In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import pickle
from celloracle.applications import Gradient_calculator
from celloracle.applications import Oracle_development_module


In [ ]:
import celloracle as co
co.__version__

In [ ]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [ ]:
Sample = "MSM144"

print("Start Analysis for Sample: " + Sample)

#First define directories
projDir = os.path.join('SCENIC/GRNind/')
dataDir = os.path.join('SCENIC/RNA/',Sample+'/')
dataFig = os.path.join(projDir+'figures_CO/')
tmpDir = "TMP/"
os.chdir(projDir)

In [ ]:
links = co.load_hdf5(dataDir+"links.celloracle.links")

In [ ]:
##laod processed data KO data
goi="CRX"
oracle = co.load_hdf5(dataDir+"output_"+goi+"KO.celloracle.oracle")


In [ ]:
# n_grid = 40 is a good starting value.
n_grid = 40
oracle.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=200)

In [ ]:
oracle.suggest_mass_thresholds(n_suggestion=12)


In [ ]:
min_mass = 26 #29 for MSM109 26 for MSM057 and MSM144
oracle.calculate_mass_filter(min_mass=min_mass, plot=True)

In [ ]:
# Visualize pseudotime
#fig, ax = plt.subplots(figsize=[6,6])
#sc.pl.embedding(adata=oracle.adata, basis=oracle.embedding_name, ax=ax, cmap="plasma",
#                color=["DPTval"])

In [ ]:
##to find simualtion
fig, ax = plt.subplots(1, 2,  figsize=[13, 6])

scale_simulation = 5
# Show quiver plot
oracle.plot_simulation_flow_on_grid(scale=scale_simulation, ax=ax[0])
ax[0].set_title(f"Simulated cell identity shift vector: {goi} KO")

# Show quiver plot that was calculated with randomized graph.
oracle.plot_simulation_flow_random_on_grid(scale=scale_simulation, ax=ax[1])
ax[1].set_title(f"Randomized simulation vector")

plt.show()

In [ ]:
# Instantiate Gradient calculator object
gradient = Gradient_calculator(oracle_object=oracle, pseudotime_key="DPTval")
#We already know approproate values for them. Please set the same values as  above.

gradient.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=200)
gradient.calculate_mass_filter(min_mass=min_mass, plot=True)

In [ ]:
#we convert the pseudotime data into grid points.
gradient.transfer_data_into_grid(args={"method": "polynomial", "n_poly":3}, plot=True)


In [ ]:
# Calculate graddient
gradient.calculate_gradient()

# Show results
scale_dev = 40
gradient.visualize_results(scale=scale_dev, s=5)

In [ ]:
# Make Oracle_development_module to compare two vector field
dev = Oracle_development_module()
# Load development flow
dev.load_differentiation_reference_data(gradient_object=gradient)
# Load simulation result
dev.load_perturb_simulation_data(oracle_object=oracle)


# Calculate inner produc scores
dev.calculate_inner_product()
dev.calculate_digitized_ip(n_bins=10)

In [ ]:
# Show perturbation scores
vm = 0.15

fig, ax = plt.subplots(1, 2, figsize=[12, 6])
dev.plot_inner_product_on_grid(vm=0.02, s=50, ax=ax[0])
ax[0].set_title(f"PS")

dev.plot_inner_product_random_on_grid(vm=vm, s=50, ax=ax[1])
ax[1].set_title(f"PS calculated with Randomized simulation vector")
plt.show()

In [ ]:
# Let's visualize the results
scale_simulation=5
p=dev.visualize_development_module_layout_0(s=5,
                                          scale_for_simulation=scale_simulation,
                                          s_grid=50,
                                          scale_for_pseudotime=scale_dev,
                                          vm=vm, return_fig=True)

In [ ]:
p.savefig(dataFig+Sample+'_'+goi+"KO.pdf")